In [3]:
def train():
    import os.path as path
    from pickle import load, dump
    import warnings

    import pandas as pd
    import numpy as np

    from sklearn import metrics
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from xgboost import XGBClassifier

    from urllib.parse import urlparse
    import mlflow
    import mlflow.sklearn

    import logging

    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def model_input():
        model = input("\n============Give model name, xgBoost.pkl, GradientBoosting.pkl, randomForest.pkl==============\n")
        return str(model)

    def eval_score(X_test):
        """
        X_test : test dataset, y_pred : prediction made,  : name of the model desired
        return score of the model
        """
        # clf = load_model(model_input())
        clf = load_model()
        preds = make_prediction(X_test)
        return metrics.accuracy_score(X_test, preds)

    def make_prediction(X_test):
        clf = load_model()
        return clf.predict(X_test)

    def load_model():
        """
         : name of the model desired
        return the model desired
        """
        model = model_input
        return load(open(path.join('6_models', "{}").format("xgBoost.pkl"), 'rb'))


    if __name__ == "__main__":
        """
        main loop to train, predict and score our models
        """
        # model = model_input()

        warnings.filterwarnings("ignore")
        np.random.seed(40)

        print("\nInitializing the program\n...")

        X_test = pd.read_csv(path.join("3_X_fitted_dataframe", "X_test_scaled.csv"))
        X_train = pd.read_csv(path.join("3_X_fitted_dataframe", "X_train_scaled.csv"))
        y_train = pd.read_csv(path.join('4_y_dataframe', 'y_train.csv'))   
        y_test = pd.read_csv(path.join('4_y_dataframe', 'y_test.csv'))   

        try:
            print("\nData loaded\n...")
        except Exception as e:
            logger.exception(
                "Unable to download training & test CSV, check your internet connection. Error: %s", e
            )

        with mlflow.start_run():
            clf = load_model()
            clf.fit(X_train, y_train)

            predicted = clf.predict(X_test)

            score = eval_score(X_test)
            mlflow.log_param("accuracy", score)
            print("\nlogged accuracy\n...")

            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.sklearn.log_model(clf, "model", registered_=str())
                print("\nmodel registered\n <3")
            else:
                mlflow.sklearn.log_model(clf, "model")

In [4]:
train()


Data loaded

[19:54:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
